In [1]:
import streamlit as st
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("Export_TER_juin2023_FIX_SafwanChendeb.csv",sep=";")
print(df.info)

<bound method DataFrame.info of          N° ENREG Code CAF  ASS_FRQ_KHZ  ASS_LGBD_KHZ  TER_ANT_AZM_MAX  \
0       1028247.0        K     38577000       14000.0             90.0   
1       1033878.0        K     19376875       13700.0            204.5   
2       1428858.0        K     26145000       56000.0            320.7   
3       1502495.0        K      6920000       40000.0            307.4   
4        339609.0        K     38552500        7000.0            319.9   
...           ...      ...          ...           ...              ...   
823645  1419579.0        K     22128750       28000.0            195.1   
823646   124559.0        J         3398           0.0              NaN   
823647  1111112.0        K     23241750       14000.0            182.8   
823648  1508178.0        K     26173000      112000.0            156.6   
823649  1276551.0        K     38570000       56000.0            168.4   

        TER_ANT_ANG  PT_LONG_DEG  PT_LONG_MIN  PT_LONG_SEC PT_LONG_ORIENT  \
0 

DATA CLEAN

Enlever les valeurs nulles ou trop petites de Largeur de bande

In [3]:
df2 = df.dropna(subset=['ASS_LGBD_KHZ'])
df2 = df2[df2["ASS_LGBD_KHZ"] >1000]


Enlever les antennes qui ont été supprimées

In [14]:
enreg_to_supp = df2[df2['MVT_CODE'] == 'SUP']['N° ENREG']
df2 = df2[~df2['N° ENREG'].isin(enreg_to_supp)]


Garder seulement les ajouts d'antenne

In [4]:
df2  = df2[df2['MVT_CODE'] == 'ADD']

Garder seulement les dernières modifications de chaque antenne

In [15]:
df2['Date CAF'] = pd.to_datetime(df['Date CAF'], format='%d/%m/%Y')
df2 = df2.sort_values('Date CAF').drop_duplicates(subset = "N° ENREG", keep = 'last')

TEST

In [7]:
df2.head(20)

,N° ENREG,Code CAF,ASS_FRQ_KHZ,ASS_LGBD_KHZ,TER_ANT_AZM_MAX,TER_ANT_ANG,PT_LONG_DEG,PT_LONG_MIN,PT_LONG_SEC,PT_LONG_ORIENT,PT_LAT_DEG,PT_LAT_MIN,PT_LAT_SEC,PT_LAT_ORIENT,Date CAF,BASE,MVT_CODE
2,1428858.0,K,26145000,56000.0,320.7,2.6,7,3,5,E,43,37,2,N,18/05/2017,H,ADD
3,1502495.0,K,6920000,40000.0,307.4,3.0,6,36,33,E,47,38,35,N,11/04/2019,R,ADD
5,1455698.0,K,6460000,40000.0,247.6,2.5,5,38,38,E,43,41,17,N,22/02/2018,H,ADD
10,1277365.0,K,13087000,28000.0,149.3,4.7,2,25,38,E,43,47,44,N,18/12/2014,H,ADD
17,1083711.0,K,6660000,40000.0,188.8,2.5,1,48,34,E,50,4,53,N,20/04/2010,H,ADD
22,1632137.0,K,19150000,110000.0,30.1,1.2,2,39,41,E,47,41,41,N,17/03/2022,R,ADD
29,1659722.0,K,81375000,500000.0,247.7,0.8,0,21,50,W,46,14,36,N,16/03/2023,R,ADD
32,1661516.0,K,19480000,110000.0,253.7,1.2,2,30,44,E,49,54,15,N,22/06/2023,R,ADD
33,1444549.0,K,19315000,55000.0,317.5,3.3,0,37,51,W,44,47,30,N,16/11/2017,H,ADD
36,1006279.0,K,22125250,7000.0,50.6,2.8,0,54,54,W,44,36,34,N,03/05/2007,H,ADD


In [12]:
import plotly.express as px

# Convert 'Date CAF' to datetime
df2['Date CAF'] = pd.to_datetime(df2['Date CAF'], format='%d/%m/%Y')

# Extract month and year and count the occurrences
df2['YearMonth'] = df2['Date CAF'].dt.to_period('M')
monthly_counts = df2['YearMonth'].value_counts().sort_index().reset_index()
monthly_counts.columns = ['YearMonth', 'Number of Antennas']

# Converting 'YearMonth' to string to avoid serialization issue
monthly_counts['YearMonth'] = monthly_counts['YearMonth'].astype(str)

# Plotting the evolution using Plotly
fig = px.line(
    monthly_counts, 
    x='YearMonth', 
    y='Number of Antennas', 
    title='Évolution du nombre d\'ajout d\'antennes par mois',
    markers=True,
    labels={'YearMonth': 'Mois', 'Number of Antennas': 'Nombre d\'antennes ajoutées'}
)

fig.update_layout(
    xaxis_title='Mois',
    yaxis_title='Nombre d\'antennes ajoutées',
    xaxis_tickangle=-45
)

# Saving the plot as an HTML file to display it
fig.write_html('monthly_antenna_additions.html')

# Displaying the plot link
print("Le graphique a été enregistré sous forme de fichier HTML. Vous pouvez le télécharger et le visualiser en suivant ce lien : sandbox:/mnt/data/monthly_antenna_additions.html")



Le graphique a été enregistré sous forme de fichier HTML. Vous pouvez le télécharger et le visualiser en suivant ce lien : sandbox:/mnt/data/monthly_antenna_additions.html
